In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)
self = features_class

In [ ]:
df_item = features_class.df_item.copy()
df_unit = features_class.df_unit.copy()

In [ ]:

feature_name = 'f__sequence_jump'
score_name = 's__sequence_jump'
df = df_item[~pd.isnull(df_item[feature_name])].copy()
# Select only those variables that have at least three distinct values and more than one hundred records
variables = self.filter_variable_name_by_frequency(df, feature_name, frequency=100, min_unique_values=3)
#pivot_table, index_col = self.get_clean_pivot_table(feature_name,remove_low_freq_col=True)

In [ ]:
for index_range in range(0, len(variables), 50):
    plt.figure(figsize=(15, 6))
    sns.boxplot(df[df['variable_name'].isin(variables[index_range:index_range+50])],x='variable_name', y=feature_name)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
from pyod.models.lunar import LUNAR
from pyod.models.rgraph import RGraph
from pyod.models.inne import INNE
# Train and predict the answer_changed alongside the variable_name. Here it uses the qnr_seq as it's a numeric variable that maps the variable name into numeric fied
model = INNE()
df[score_name] = model.fit_predict(df[['qnr_seq',feature_name]])

In [ ]:
df.columns

In [ ]:
df.groupby('variable_name')[score_name].value_counts()#/df[score_name].count()

In [ ]:
for var in variables:
    mask = (df['variable_name']==var) 
    mask1 = (df[feature_name]!=0) 
    model = INNE()
    df.loc[mask,score_name] = model.fit_predict(df[mask][[feature_name]])
    if df[(df[feature_name]==0)&mask][score_name].sum()>0:
        print(df[mask][feature_name].value_counts())
    bins = np.histogram_bin_edges(df[mask&mask1][feature_name], bins=12)
    no_anomaly_df = df[(df[score_name]==0)&mask&mask1][feature_name]
    anomaly_df = df[(df[score_name]==1)&mask&mask1][feature_name]
    
    plt.hist(no_anomaly_df, bins=bins, alpha=0.5, color='blue', label='True')
    plt.hist(anomaly_df, bins=bins, alpha=0.5, color='red', label='False')
    plt.title(var)
    plt.show()

In [ ]:
    plt.bar(no_anomaly_df['responsible'], no_anomaly_df[feature_name], color='blue', label='No Anomaly')
    plt.bar(anomaly_df['responsible'], anomaly_df[feature_name], color='red', label='Anomaly')
    plt.xticks(rotation=90)
    plt.title(var)
    plt.show()